# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [2]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [3]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [4]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [5]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

12


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [6]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [7]:
# Create a Keyspace schema
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS musicdb
    WITH REPLICATION = 
    { 'class': 'SimpleStrategy', 'replication_factor' : 1}"""    
)
except Exception as e:
    print(e)

In [8]:
# Set Keyspace 
try:
    session.set_keyspace('musicdb')
except Exception as e:
    print(e)

### Create and load the event data into the event_data table.

In [10]:
# FOR THIS QUERY, SESSIONID AND ITEMINSESSION ARE CONSIDERED AS THE COMPOSITE KEY BECAUSE DATA WILL BE FILTERED BY SESSIONID AND ITEMINSESSION BASED ON THE REQUIREMENT.

query = "CREATE TABLE IF NOT EXISTS event_data "
query = query + "(sessionId int, itemInSession int, artist text, firstName varchar, gender varchar, lastName varchar, length float, level varchar, location text, song text, userId int, \
                  PRIMARY KEY(sessionId, itemInSession))"
try:
    session.execute(query)    
except Exception as e:
    print(e)
    
    
# Read and load the csv data into the table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
# Load data into event data table        
        query = "INSERT INTO event_data(sessionId, itemInSession, artist, firstName, gender, lastName, length, level, location, song, userid) "
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[1], line[2], line[4], float(line[5]), line[6], line[7], line[9], int(line[10])))

### Verifying data inserted into the event_data table 

In [11]:
# Retrieve the event data when sessionID = 338 and itemInSession = 4
query = "SELECT artist, song, length FROM event_data where sessionId = 338 and itemInSession = 4"

try:
    rows = session.execute(query)    
except Exception as e:
    print(e)
    
for row in rows:
    print(row.artist, row.firstname, row.gender, row.iteminsession, row.lastname, row.length, row.level, row.location, row.sessionid)

### Create and load playlist data into playlist_data table

In [12]:
# FOR THIS QUERY, USERID, SESSIONID AND ITEMINSESSION ARE CONSIDERED AS THE COMPOSITE KEY BECAUSE DATA WILL BE QUERIED BY USERID OR SESSIONID AND ITEMINSESSION BASED ON THE REQUIREMENT.

query = "CREATE TABLE IF NOT EXISTS playlist_data "
query = query + "(artist text, song text, firstName varchar, lastName varchar, sessionId int, userId int,  itemInSession int, \
                  PRIMARY KEY((userId,sessionId), itemInSession))"
try:
    session.execute(query)    
except Exception as e:
    print(e)
    
# Read and load the csv data into the table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
# Load data into the playlist data table      
        query = "INSERT INTO playlist_data(artist, song, firstName, lastName, sessionId, userId, itemInSession) "
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (line[0], line[9], line[1], line[4], int(line[8]), int(line[10]), int(line[3])))

### Verifying data inserted into the playlist_data table 

In [13]:
query = "SELECT artist, song, firstName, lastName FROM playlist_data WHERE userid = 26 and sessionid = 169" 

try:
    rows = session.execute(query)    
except Exception as e:
    print(e)
    
for row in rows:
    print(row.artist, row.song, row.firstname, row.lastname)

Fall Out Boy Nobody Puts Baby In The Corner Ryan Smith
M.I.A. Mango Pickle Down River (With The Wilcannia Mob) Ryan Smith


### Create and load user data into user_data table

In [14]:
# FOR THIS QUERY, SONG AND USERID ARE CONSIDERED AS THE COMPOSITE KEY BECAUSE DATA WILL BE FILTERED BY SONG AND USERID BASED ON THE REQUIREMENT.

query = "CREATE TABLE IF NOT EXISTS user_data "  
query = query + "(firstName varchar, lastName varchar, song text, userId int, PRIMARY KEY(song, userId))"
try:
    session.execute(query)    
except Exception as e:
    print(e)

# Read and load the csv data into the table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
# Load data into the user data table  
        query = "INSERT INTO user_data(firstName, lastName, song, userId) "
        query = query + "VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[1], line[4], line[9], int(line[10])))


### Verifying data inserted into the user_data table

In [15]:
query = "SELECT firstName, lastName, song FROM user_data WHERE song = 'You Gotta Be'"

try:
    rows = session.execute(query)    
except Exception as e:
    print(e)
    
for row in rows:
    print(row.firstname, row.lastname, row.song)

Kaylee Summers You Gotta Be


### Drop the tables before closing out the sessions

In [16]:
# Drop event data table
query = 'drop table event_data'
try:
    session.execute(query)    
except Exception as e:
    print(e)

# Drop playlist data table
query = 'drop table playlist_data'    
try:
    session.execute(query)    
except Exception as e:
    print(e) 

# Drop user data table    
query = 'drop table user_data'
try:
    session.execute(query)    
except Exception as e:
    print(e) 

### Close the session and cluster connection¶

In [17]:
session.shutdown()
cluster.shutdown()